In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
from IPython.display import clear_output, FileLink, FileLinks
import collections
import urllib
from os import listdir
from os.path import isfile, join
import pandas
import re

In [13]:
raw_x = np.sin(range(0,100000,1))

In [52]:
Dimensions = collections.namedtuple("Dimensions","num_neurons, num_steps, batch_size")
dimensions = Dimensions(num_neurons = 500,
                        num_steps = 10,
                        batch_size = 10*128)

In [53]:
def gen_batch(raw_x, batch_size, num_steps):

    if batch_size % num_steps != 0:
        warnings.warn("Warning: batch size is not a multiple of num_steps.")
    
    raw_y = raw_x[1:] + [raw_x[0]]
    
    # partition raw data into batches and stack them vertically in a data matrix
    num_batch = len(raw_x) // batch_size
    data_x = np.zeros([num_batch, batch_size], dtype=np.int32)
    data_y = np.zeros([num_batch, batch_size], dtype=np.int32)
    
    for i in range(num_batch):
        data_x[i] = raw_x[batch_size * i:batch_size * (i + 1)]
        data_y[i] = raw_y[batch_size * i:batch_size * (i + 1)]
    
    # further divide batch partitions into num_steps for truncated backprop
    batch_lines = batch_size // num_steps

    for i in range(num_batch):
        x = np.zeros([batch_lines,num_steps],dtype=np.int32)
        y = np.zeros([batch_lines,num_steps],dtype=np.int32)
        for n in range(batch_lines):
            x[n,:] = data_x[i, n * num_steps:(n + 1) * num_steps]
            y[n,:] = data_y[i, n * num_steps:(n + 1) * num_steps]
        yield (x, y)

In [54]:
def gen_epochs(raw_data, num_epochs, batch_size, num_steps):
    for i in range(num_epochs):
        yield gen_batch(raw_data,batch_size,num_steps)

In [55]:
GraphOps = collections.namedtuple("GraphOps","x y learning_rate train prediction init_state state accuracy loss init")

In [81]:
def build_graph(dims):    
    
    tf.reset_default_graph()
    
    #----------input layer-----------------------
    x = tf.placeholder(tf.float32, name="x")
    y = tf.placeholder(tf.float32, name="y")
    #init_state = tf.placeholder(tf.float32,[None, num_neurons])
    learning_rate = tf.placeholder(tf.float32,[])

    #embeddings = tf.get_variable('embedding_matrix', [dims.num_steps, dims.num_neurons])

    # Tensor of dims batch_size //num_steps x num_steps x num_neurons
    #x_transformed = tf.nn.embedding_lookup(embeddings, x)
    
    #x_transformed = tf.reshape(x, [dims.batch_size //dims.num_steps , dims.num_steps])

    #----------hidden layer-----------------------
    cell = tf.nn.rnn_cell.LSTMCell(dims.num_neurons, state_is_tuple=True)
    cell = tf.nn.rnn_cell.MultiRNNCell([cell] * 3, state_is_tuple=True)
    init_state = cell.zero_state(dims.batch_size//dims.num_steps,tf.float32)
    #rnn_output, final_state = tf.nn.dynamic_rnn(cell, x_transformed, initial_state=init_state, dtype=tf.float32)
    rnn_output, final_state = tf.nn.rnn(cell, x, init_state)
    
    #https://github.com/rajshah4/tensorflow_shiny/blob/master/RNN_demo.ipynb

    #----------output layer-----------------------
    #weight = tf.Variable(tf.truncated_normal([dims.num_neurons, dims.vocab_size], stddev=0.1))
    weight = tf.Variable(tf.truncated_normal([dims.num_neurons, 1], stddev=0.1))
    #bias = tf.Variable(tf.constant(0.0, shape=[dims.vocab_size]))
    bias = tf.Variable(tf.constant(0.0, shape=[1]))
    
    output = tf.reshape(rnn_output, [-1, dims.num_neurons])
    value = tf.matmul(output, weight) + bias
    #prediction = tf.nn.softmax(logits)
    prediction = value
    #prediction = tf.reshape(prediction, [-1, dims.num_steps, dims.vocab_size])

    #----------loss and accuracy------------------
    target = y
    #loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits,target))
    loss = tf.sqrt(tf.reduce_mean(tf.square(tf.sub(value, y))))
    
    accuracy = tf.reduce_mean(tf.cast(prediction - y, tf.float32))

    #----------Trainer------------------
    trainOp = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
    init_vars = tf.initialize_all_variables()
    
    return GraphOps(x=x, 
                    y=y, 
                    learning_rate=learning_rate, 
                    train=trainOp, 
                    prediction=prediction, 
                    init_state=init_state, 
                    state=final_state, 
                    accuracy=accuracy, 
                    loss=loss, 
                    init=init_vars)

In [82]:
print_after_num_steps = 10
adaptive_learning_rate = 0.01
training_state = None

trainHist = []
accHist = []

trainingOps = build_graph(dimensions)

sess = tf.Session()
sess.run(trainingOps.init)


TypeError: inputs must be a sequence

In [66]:
for num_epoch, epoch in enumerate(gen_epochs(raw_x,5,dimensions.batch_size,dimensions.num_steps)):
    
    if num_epoch > 30:
        adaptive_learning_rate = 0.001
    
    for i, batch in enumerate(epoch):
        
        feed_dic = {trainingOps.x: batch[0], trainingOps.y: batch[1], trainingOps.learning_rate: adaptive_learning_rate}
        if training_state is not None:
            feed_dic = {trainingOps.x: batch[0], trainingOps.y: batch[1], trainingOps.learning_rate: adaptive_learning_rate, trainingOps.init_state: training_state}
        acc, lossVal, training_state, _ = sess.run([trainingOps.accuracy, trainingOps.loss, trainingOps.state, trainingOps.train], feed_dict=feed_dic)
        trainHist.append(lossVal)
        accHist.append(acc)
        
        if i % print_after_num_steps == 0:
            clear_output()
            avg_loss = sum(trainHist[-print_after_num_steps:])/print_after_num_steps
            avg_acc = sum(accHist[-print_after_num_steps:])/print_after_num_steps
            logstring = 'Epoch: '+ str(num_epoch) + ' step: ' + str(i) +' loss: ' + str(avg_loss) + ' accuracy: ' + str(avg_acc) + '\n'
            print(logstring)
            with open("name_gen_log.txt", "a") as myfile:
                myfile.write(logstring)

Epoch: 1 step: 60 loss: nan accuracy: nan



KeyboardInterrupt: 

In [27]:
saver = tf.train.Saver()
saver.save(sess, 'lstm_rnn_m_name_gen.tf_vars')
FileLink('lstm_rnn_m_name_gen.tf_vars')

ValueError: No variables to save